In [ ]:
import pandas as pd
import gower
import numpy as np
from sklearn.cluster import AffinityPropagation
from sklearn import metrics

In [2]:
columns = ['estimatedp', 'wtr_srcdes', 'wtr_hauled', 'priv_wells', \
               'wc_exists', 'wc_adeq', 'wc_hlth', 'ww_public', 'ppl_nowat_r', 'ppl_noww_r', 'ppl_yeswat_r', 'ppl_yesww_r']
colonias_Y_path = 'dataset/colonias_Y_norm.csv'
colonias_N_path = 'dataset/colonias_N_norm.csv'

In [3]:
df = pd.read_csv(colonias_Y_path)[columns]
selected_df = df[(df['ppl_nowat_r'] <= 1) & (df['ppl_noww_r'] <= 1)
                 & (df['ppl_yeswat_r'] <= 1) & (df['ppl_yesww_r'] <= 1)]

similarity_metric = gower.gower_matrix(selected_df)
preference = np.median(similarity_metric)
np.fill_diagonal(similarity_metric, preference)
similarity_metric *= -1

In [6]:
"""
Colonias_N:
Best params: Damping= 0.6 , iteration= 500  n_clusters= 11 The average SScore= 0.6063733

Colonias_Y:
Best params: Damping= 0.6 , iteration= 300  n_clusters= 24 The average SScore= 0.8435476
"""
af = AffinityPropagation(affinity='precomputed', damping = 0.6, max_iter=300).fit(similarity_metric)
n_clusters = len(np.unique(af.labels_))
cluster_labels = af.labels_
print(pd.DataFrame(af.labels_).value_counts())
print("num clusters:", n_clusters)
print(metrics.silhouette_score(similarity_metric, cluster_labels))

with open('dataset/colonias_Y_label.csv', 'w') as f:
    for label in cluster_labels:
        f.write(str(label) + "\n")
f.close()


21    860
19    857
0      45
9      29
15     20
4      17
5      16
20     13
22     12
18      9
1       9
2       8
7       7
13      7
12      5
23      5
14      4
17      4
11      4
6       4
16      1
10      1
8       1
3       1
dtype: int64
num clusters: 24
0.8435476
